# 🧵 Concurrencia y Paralelismo en Python 

Este notebook cubre los principales modelos de ejecución simultánea en Python:

- Subprocesos (`subprocess`)
- Multiprocesamiento (`multiprocessing`)
- Multithreading (`threading`, `concurrent.futures.ThreadPoolExecutor`)
- Concurrencia asíncrona (`asyncio`, `aiohttp`)

Incluye ejemplos detallados y ejercicios prácticos con solución.

---


## ✅ Subprocesos externos con `subprocess` – Ejemplo


In [4]:
import subprocess

# Ejecuta un comando externo del sistema operativo (solo Unix/Linux)
out = subprocess.check_output(['uname', '-v'], text=True)
print(out.strip())


#65-Ubuntu SMP PREEMPT_DYNAMIC Mon May 19 17:15:03 UTC 2025


## 📝 Ejercicio 1: Lista los archivos del directorio actual usando subprocess


In [5]:
salida = subprocess.check_output(['ls', '-l'], text=True)
print(salida)


total 8
-rw-rw-r-- 1 ags ags 5492 jul  1 01:27 M500_concurrencia.ipynb



## ✅ Paralelismo con `multiprocessing.Pool` (CPU-bound) – Ejemplo


In [6]:
from multiprocessing import Pool

def cuadrado(x):
    return x * x

# Ejecuta varios procesos en paralelo
with Pool(processes=4) as pool:
    resultados = pool.map(cuadrado, range(10))
    print(resultados)


[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


## 📝 Ejercicio 2: Calcula el cubo de los números del 1 al 5 usando Pool


In [7]:
def cubo(x):
    return x ** 3

with Pool(processes=2) as pool:
    resultados_cubo = pool.map(cubo, range(1, 6))
    print(resultados_cubo)


[1, 8, 27, 64, 125]


## ✅ Concurrencia con `threading` (I/O-bound) – Ejemplo


In [22]:
import threading
import time

def leer_archivo_simulado(nombre_archivo):
    print(f'📂 Abriendo {nombre_archivo}...')
    time.sleep(1)  # Simula retardo de I/O (ej: leer un archivo grande)
    print(f'✅ Terminado {nombre_archivo}')

nombres_archivos = [f"archivo_{i}.txt" for i in range(3)]
hilos = []

inicio = time.perf_counter()  # Medimos el tiempo inicial

for nombre in nombres_archivos:
    hilo = threading.Thread(target=leer_archivo_simulado, args=(nombre,))
    hilos.append(hilo)
    hilo.start()

for hilo in hilos:
    hilo.join()

fin = time.perf_counter()  # Tiempo final

print(f"⏱️ Tiempo total: {fin - inicio:.2f} segundos")


📂 Abriendo archivo_0.txt...
📂 Abriendo archivo_1.txt...
📂 Abriendo archivo_2.txt...
✅ Terminado archivo_0.txt
✅ Terminado archivo_1.txt
✅ Terminado archivo_2.txt
⏱️ Tiempo total: 1.00 segundos


## 📝 Ejercicio 3: Crea 5 hilos que impriman su número después de 0.5 segundos


In [9]:
def escribir_num(n):
    time.sleep(0.5)
    print(f'Hilo {n} ejecutado')

hilos = [threading.Thread(target=escribir_num, args=(i,)) for i in range(5)]
for h in hilos:
    h.start()
for h in hilos:
    h.join()


Hilo 0 ejecutado
Hilo 2 ejecutado
Hilo 4 ejecutado
Hilo 1 ejecutado
Hilo 3 ejecutado


## ✅ Concurrencia asíncrona con `asyncio` + `aiohttp` – Ejemplo


In [3]:
import asyncio
import aiohttp

async def fetch(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            contenido = await response.text()
            return len(contenido)

async def main():
    url = 'https://example.com'
    tamanio = await fetch(url)
    print(f'📄 Descargado {tamanio} caracteres de {url}')

# Importante: Solo usa "await main()" dentro de otros entornos async.
# Si ejecutas en notebook normal:
await main()


📄 Descargado 1256 caracteres de https://example.com


## 📝 Ejercicio 4: Descargar 3 URLs diferentes en paralelo y medir el tiempo total


In [11]:
import time

async def fetch_con_tiempo(url):
    inicio = time.perf_counter()
    await fetch(url)
    fin = time.perf_counter()
    print(f'{url} descargada en {fin - inicio:.2f} segundos')

async def main_ejercicio():
    urls = ['https://example.com', 'https://httpbin.org/get', 'https://www.python.org']
    await asyncio.gather(*(fetch_con_tiempo(u) for u in urls))

# asyncio.run(main_ejercicio())
await main_ejercicio()


https://www.python.org descargada en 0.10 segundos
https://example.com descargada en 0.59 segundos
https://httpbin.org/get descargada en 0.79 segundos


## ✅ Pool de hilos con `concurrent.futures.ThreadPoolExecutor` – Ejemplo


In [12]:
from concurrent.futures import ThreadPoolExecutor
import math

def cuadrado(x):
    return x ** 2

with ThreadPoolExecutor(max_workers=4) as executor:
    resultados = list(executor.map(cuadrado, range(10)))
    print(resultados)


[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


## 📝 Ejercicio 5: Calcula las raíces cuadradas de los números del 1 al 5


In [13]:
def raiz_cuadrada(x):
    return math.sqrt(x)

with ThreadPoolExecutor() as executor:
    resultados_raiz = list(executor.map(raiz_cuadrada, range(1, 6)))
    print(resultados_raiz)


[1.0, 1.4142135623730951, 1.7320508075688772, 2.0, 2.23606797749979]


## ✅ Pool de procesos con `concurrent.futures.ProcessPoolExecutor` – Ejemplo


In [14]:
from concurrent.futures import ProcessPoolExecutor

def factorial(n):
    resultado = 1
    for i in range(1, n + 1):
        resultado *= i
    return resultado

with ProcessPoolExecutor() as executor:
    resultados = list(executor.map(factorial, range(1, 6)))
    print(resultados)


[1, 2, 6, 24, 120]


## 📝 Ejercicio 6: Calcula los cuadrados de 1 a 5 usando ProcessPoolExecutor


In [15]:
def cuadrado(x):
    return x * x

with ProcessPoolExecutor() as executor:
    resultados_cuadrados = list(executor.map(cuadrado, range(1, 6)))
    print(resultados_cuadrados)


[1, 4, 9, 16, 25]


## ✅ Conclusiones finales

| Técnica | Casos de uso recomendado |
|---|---|
| `subprocess` | Ejecutar comandos externos (ej: sistema operativo) |
| `threading` | Tareas I/O-bound bloqueantes (red, disco) |
| `multiprocessing` / `ProcessPoolExecutor` | Cálculos CPU-bound que escalan con múltiples núcleos |
| `ThreadPoolExecutor` | Simplificar uso de hilos en código I/O |
| `asyncio` | Miles de tareas I/O concurrentes y ligeras |

➡️ La elección depende del tipo de tarea: **I/O-bound vs CPU-bound**, y si buscas **concurrencia** o **paralelismo real**.

---


## 📝 Ejercicio final: Comparativa de rendimiento entre Threads, Procesos y Asyncio

**Objetivo:**  
Calcular los cuadrados de los números del 1 al 10 usando tres enfoques diferentes:

1. **ThreadPoolExecutor (con hilos)**
2. **ProcessPoolExecutor (con procesos)**
3. **asyncio (con una función simulada que hace un `asyncio.sleep`)**

**Además:**  
- Mide el tiempo total de ejecución en cada caso usando `time.perf_counter()`.
- Compara los resultados y discute cuál es más rápido y por qué.

---


In [21]:
import time
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import asyncio

def cuadrado(x):
    time.sleep(1)  # Simula una tarea que tarda un poco
    return x * x

# 1. Usando ThreadPoolExecutor
start = time.perf_counter()
with ThreadPoolExecutor() as executor:
    resultados_thread = list(executor.map(cuadrado, range(1, 11)))
end = time.perf_counter()
print(f"ThreadPoolExecutor → {resultados_thread} en {end - start:.2f} segundos")

# 2. Usando ProcessPoolExecutor
start = time.perf_counter()
with ProcessPoolExecutor() as executor:
    resultados_process = list(executor.map(cuadrado, range(1, 11)))
end = time.perf_counter()
print(f"ProcessPoolExecutor → {resultados_process} en {end - start:.2f} segundos")

# 3. Usando asyncio
async def cuadrado_async(x):
    await asyncio.sleep(1)  # Simula retardo asíncrono
    return x * x

async def main_async():
    tareas = [cuadrado_async(x) for x in range(1, 11)]
    return await asyncio.gather(*tareas)

start = time.perf_counter()
resultados_async = await main_async()
end = time.perf_counter()
print(f"Asyncio → {resultados_async} en {end - start:.2f} segundos")


ThreadPoolExecutor → [1, 4, 9, 16, 25, 36, 49, 64, 81, 100] en 1.00 segundos
ProcessPoolExecutor → [1, 4, 9, 16, 25, 36, 49, 64, 81, 100] en 1.09 segundos
Asyncio → [1, 4, 9, 16, 25, 36, 49, 64, 81, 100] en 1.00 segundos
